### DENGUE AI : Predicting Disease Spread By www.drivendata.org 

I chose this dataset to answer few questions:

1) Normally reported number of cases

2) Optimal conditions i.e temperature and NDVI scale value for dengue spread

3) At what time of year this is most common

4) How standard variables are related to those measured by satellites and those measured locally?


In [1]:
import numpy as np
import pandas as pd
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.figure_factory as ff
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [2]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in '%s': %s" % (cwd, files))

Files in 'C:\Users\ACER M5\Dengue Project Data': ['.ipynb_checkpoints', 'Dengue_Data_Model.ipynb', 'dengue_features_test.csv', 'dengue_features_train.csv', 'dengue_labels_train.csv', 'logs', 'submission_1.csv', 'submission_2.csv', 'submission_format.csv']


In [3]:
raw_data_input=pd.read_csv('dengue_features_train.csv')

In [4]:
raw_data_input.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [5]:
raw_data_input.columns

Index(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm'],
      dtype='object')

In [6]:
raw_data_targets=pd.read_csv('dengue_labels_train.csv')

In [7]:
raw_data_targets.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [8]:
raw_data_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 24 columns):
city                                     1456 non-null object
year                                     1456 non-null int64
weekofyear                               1456 non-null int64
week_start_date                          1456 non-null object
ndvi_ne                                  1262 non-null float64
ndvi_nw                                  1404 non-null float64
ndvi_se                                  1434 non-null float64
ndvi_sw                                  1434 non-null float64
precipitation_amt_mm                     1443 non-null float64
reanalysis_air_temp_k                    1446 non-null float64
reanalysis_avg_temp_k                    1446 non-null float64
reanalysis_dew_point_temp_k              1446 non-null float64
reanalysis_max_air_temp_k                1446 non-null float64
reanalysis_min_air_temp_k                1446 non-null float64
reanalysis_precip

In [9]:
columns=raw_data_input.describe().columns
for c in columns:
    raw_data_input[c].fillna(raw_data_input[c].median(),inplace=True)

In [10]:
raw_data_input['total_cases']=raw_data_targets['total_cases']
raw_data=raw_data_input

In [11]:
corr_matrix=raw_data.corr()
corr_matrix['total_cases'].sort_values()

year                                    -0.306806
reanalysis_tdtr_k                       -0.277997
station_diur_temp_rng_c                 -0.233778
reanalysis_max_air_temp_k               -0.191131
ndvi_ne                                 -0.164899
ndvi_sw                                 -0.147349
ndvi_nw                                 -0.142665
reanalysis_relative_humidity_percent    -0.132336
ndvi_se                                 -0.124698
station_precip_mm                       -0.073159
station_max_temp_c                      -0.039317
precipitation_amt_mm                    -0.038902
reanalysis_sat_precip_amt_mm            -0.038902
reanalysis_precip_amt_kg_per_m2         -0.010137
station_avg_temp_c                       0.114268
reanalysis_specific_humidity_g_per_kg    0.129741
reanalysis_dew_point_temp_k              0.142397
reanalysis_avg_temp_k                    0.151435
weekofyear                               0.216452
reanalysis_air_temp_k                    0.264546


In [12]:
corr_matrix.iplot(kind='heatmap',colorscale='spectral',size=(10,12))

### Most of the variables are negatively correlated with total_cases apart from few which are slightly positive but are mostly centered around 0 which means no correlation

Here we can see that many variables are highly correlated with each other such as 
1. precipitation_amt_mm and reanalysis_sat_amt_mm
2. reanalysis_min_air_temp and reananlysis_tdtr_k
3. reanalysis_max_air_temp and reanalysis_tdtr_k

We can remove one of these along with all those who little or zero correlation with total cases as seen in the figure such as 

station_precip_mm                      -0.073159

station_max_temp_c                      -0.039317

precipitation_amt_mm                    -0.038902

reanalysis_sat_precip_amt_mm            -0.038902

reanalysis_precip_amt_kg_per_m2         -0.010137

Hence excluded columns are shown along with there correlation coeffecient.

In [13]:
excluded_columns=['station_precip_mm','station_max_temp_c','precipitation_amt_mm','reanalysis_sat_precip_amt_mm','reanalysis_precip_amt_kg_per_m2']
raw_data=raw_data_input[set(raw_data_input.columns)-set(excluded_columns)]

### Now Lets start to visualize this data through vector graphics of PLOTLY.JS which is used with pandas dataframe through cufflinks.

In [14]:
raw_data['total_cases'].iplot(kind='histogram',histnorm='density')

### Since the cases are mentioned for each year and city, so as this graph tells us most occurences are of 0-10 cases for different cities.

Also this shows skewness in output variable that is positive skewness so we should take log or sqrt to normalize it. 

In [15]:
raw_data['total_cases']=np.log(raw_data['total_cases']+1)
raw_data['total_cases'].iplot(kind='histogram',histnorm='density')

In [16]:
for i in range (raw_data.shape[0]):
    if raw_data['total_cases'][i] < 0.9596:
          raw_data.drop(index=i,inplace=True)
          raw_data_targets.drop(index=i,inplace=True)

In [17]:
raw_data['total_cases'].iplot(kind='histogram',histnorm='density')

In [18]:
attributes_1=['ndvi_ne','ndvi_nw','ndvi_sw','ndvi_se']
raw_data[attributes_1].iplot(filename='cufflinks/line-example')


### ndvi is a method to measure vegitation by geograohical survey. -1 means water and +1 means greenery while around zero means that it is urban area. Here since all four indictors of that are around zero and northeast and northwest are more inclined towards negative side (water) and southeast and southwest are more on positive side (green vegetation)

This means that this locality is urban area with northern part more having water and southern part having more greenery.

In [19]:
attributes_3=['reanalysis_max_air_temp_k','reanalysis_air_temp_k','reanalysis_avg_temp_k','reanalysis_min_air_temp_k']
raw_data[attributes_3].iplot(kind='box')

### The average air and normal temperature are both centered around 295-300K which means this is the ideal temperature for dengue cases

In [20]:
raw_data['station_avg_temp_k']=raw_data['station_avg_temp_c'] + 273.13
attributes_4=['station_avg_temp_k','reanalysis_avg_temp_k']
raw_data[attributes_4].iplot(kind='line')

### Reanalysis is a systematic approach for measuring climate changes regularly over 6 hours iterations through the satellites and other charts. The plot here shows the difference between normal measured average temperature and from reanalysis technique.

### Preprocessing data

In [21]:
output=raw_data['total_cases']
raw_data.drop(columns='total_cases',inplace=True)

In [22]:
raw_data.drop(columns='station_avg_temp_c',inplace=True)    ### this has been changed to kelvin earlier

In [23]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
var=encoder.fit_transform(raw_data['city'])
raw_data['city']=var

In [24]:
raw_data.drop(columns='week_start_date',inplace=True)

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
scaled_data= scaler.fit_transform(raw_data)

### Linear Regression

In [26]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
Lasso_reg=Lasso(alpha=0.001,max_iter=3000)
Lasso_reg.fit(scaled_data,output)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=3000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [27]:
scores = cross_val_score(Lasso_reg, scaled_data, output, cv=10, scoring='neg_mean_absolute_error')
print(scores.mean(),scores.std())

-0.6950990768493777 0.12471527699216356


### Preparing Test Data

In [28]:
columns_req=raw_data.columns

In [29]:
test_data=pd.read_csv('dengue_features_test.csv')

In [30]:
columns=test_data.describe().columns
for c in columns:
    test_data[c].fillna(test_data[c].median(),inplace=True)

In [31]:
var1=encoder.fit_transform(test_data['city'])
test_data['city']=var1

In [32]:
test_data['station_avg_temp_k']=test_data['station_avg_temp_c'] + 273.13

In [33]:
test_data.drop(columns=['week_start_date','station_avg_temp_c'],inplace=True)

In [34]:
test_data=test_data[set(test_data.columns)-set(excluded_columns)]

In [35]:
test_data=test_data[columns_req]     ### Remember test data and input data should have same columns sequence

In [36]:
scaler_1=MinMaxScaler()
test_data=scaler_1.fit_transform(test_data)

### Lasso Reg Predictions

In [37]:
test_predictions_1=Lasso_reg.predict(test_data)

In [38]:
actual_test_predictions=np.e**(test_predictions_1 - 1)

In [39]:
submission_file=pd.read_csv('submission_format.csv')
submission_file['total_cases']=actual_test_predictions.astype(int)

In [40]:
submission_file.to_csv('submission_1.csv',index=False)

### This file scored 35 MAE on website.

## Now that we have seen a baseline result let's move to bagging and stacking

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [42]:
param_grid= {'n_estimators' : [10,100,500,1000] ,
              'max_depth' : [None,5,10,20,50],
              'oob_score' : [True,False] ,
              'bootstrap' : [True]
            }

In [43]:
Gsc= GridSearchCV(estimator=RandomForestRegressor(),param_grid=param_grid,cv=5, scoring='neg_mean_squared_error', verbose=1,n_jobs=-1)
grid_result= Gsc.fit(scaled_data,output)
best_paramters= grid_result.best_params_

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  6.1min finished
C:\Users\ACER M5\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



In [44]:
best_paramters

{'bootstrap': True, 'max_depth': 5, 'n_estimators': 100, 'oob_score': False}

In [45]:
RFRegressor=RandomForestRegressor(bootstrap=True,max_depth=5,n_estimators=1000,oob_score=False)
RFRegressor.fit(scaled_data,output)
scores_1=cross_val_score(RFRegressor,scaled_data,output,cv=10,scoring='neg_mean_absolute_error')

In [46]:
print(scores_1.mean(),scores_1.std())

-0.7122670563723514 0.20277519517973588


### The result is not better than Lasso Reg and the variance is higher than Lasso

## Let's move on to NN

In [47]:
import keras

Using TensorFlow backend.


In [48]:
from keras.layers import *
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor

In [49]:
## RUN_Name='fifth run with total 4 layers and relu,mean_absolute_error and adam'

In [54]:
## Define Model
def new_model():
    model=Sequential()
    model.add(Dense(32, input_dim=18, activation='relu', name='layer_1'))
    model.add(Dense(64,activation='relu' , name='layer_2'))
    model.add(Dense(1, activation='linear', name='output_layer'))
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

KRegressor= KerasRegressor(build_fn=new_model,batch_size=32,epochs=100)
accuracy= cross_val_score(KRegressor,scaled_data,output,cv=5,scoring='neg_mean_absolute_error')

Epoch 1/100
1037/1037 [==============================] - 9s 9ms/step - loss: 1.7217 - acc: 0.0000e+00
Epoch 2/100
1037/1037 [==============================] - 0s 261us/step - loss: 0.7240 - acc: 0.0000e+00
Epoch 3/100
1037/1037 [==============================] - 0s 279us/step - loss: 0.6905 - acc: 0.0000e+00
Epoch 4/100
1037/1037 [==============================] - 0s 293us/step - loss: 0.6832 - acc: 0.0000e+00
Epoch 5/100
1037/1037 [==============================] - 0s 267us/step - loss: 0.6822 - acc: 0.0000e+00
Epoch 6/100
1037/1037 [==============================] - 0s 262us/step - loss: 0.6777 - acc: 0.0000e+00
Epoch 7/100
1037/1037 [==============================] - 0s 290us/step - loss: 0.6694 - acc: 0.0000e+00
Epoch 8/100
1037/1037 [==============================] - 0s 284us/step - loss: 0.6673 - acc: 0.0000e+00
Epoch 9/100
1037/1037 [==============================] - 0s 298us/step - loss: 0.6668 - acc: 0.0000e+00
Epoch 10/100
1037/1037 [==============================] - 0s 269us

1037/1037 [==============================] - 0s 279us/step - loss: 0.5434 - acc: 0.0000e+00
Epoch 79/100
1037/1037 [==============================] - 0s 267us/step - loss: 0.5474 - acc: 0.0000e+00
Epoch 80/100
1037/1037 [==============================] - 0s 270us/step - loss: 0.5569 - acc: 0.0000e+00
Epoch 81/100
1037/1037 [==============================] - 0s 261us/step - loss: 0.5469 - acc: 0.0000e+00
Epoch 82/100
1037/1037 [==============================] - 0s 248us/step - loss: 0.5487 - acc: 0.0000e+00
Epoch 83/100
1037/1037 [==============================] - 0s 254us/step - loss: 0.5397 - acc: 0.0000e+00
Epoch 84/100
1037/1037 [==============================] - 0s 265us/step - loss: 0.5401 - acc: 0.0000e+00
Epoch 85/100
1037/1037 [==============================] - 0s 258us/step - loss: 0.5426 - acc: 0.0000e+00
Epoch 86/100
1037/1037 [==============================] - 0s 235us/step - loss: 0.5480 - acc: 0.0000e+00
Epoch 87/100
1037/1037 [==============================] - 0s 289us/s

1037/1037 [==============================] - 0s 235us/step - loss: 0.5477 - acc: 0.0000e+00
Epoch 56/100
1037/1037 [==============================] - 0s 226us/step - loss: 0.5469 - acc: 0.0000e+00
Epoch 57/100
1037/1037 [==============================] - 0s 214us/step - loss: 0.5458 - acc: 0.0000e+00
Epoch 58/100
1037/1037 [==============================] - 0s 203us/step - loss: 0.5437 - acc: 0.0000e+00
Epoch 59/100
1037/1037 [==============================] - 0s 235us/step - loss: 0.5477 - acc: 0.0000e+00
Epoch 60/100
1037/1037 [==============================] - 0s 221us/step - loss: 0.5572 - acc: 0.0000e+00
Epoch 61/100
1037/1037 [==============================] - 0s 220us/step - loss: 0.5540 - acc: 0.0000e+00
Epoch 62/100
1037/1037 [==============================] - 0s 209us/step - loss: 0.5483 - acc: 0.0000e+00
Epoch 63/100
1037/1037 [==============================] - 0s 229us/step - loss: 0.5688 - acc: 0.0000e+00
Epoch 64/100
1037/1037 [==============================] - 0s 239us/s

1038/1038 [==============================] - 0s 243us/step - loss: 0.5959 - acc: 0.0000e+00
Epoch 33/100
1038/1038 [==============================] - 0s 221us/step - loss: 0.5974 - acc: 0.0000e+00
Epoch 34/100
1038/1038 [==============================] - 0s 212us/step - loss: 0.5891 - acc: 0.0000e+00
Epoch 35/100
1038/1038 [==============================] - 0s 223us/step - loss: 0.5914 - acc: 0.0000e+00
Epoch 36/100
1038/1038 [==============================] - 0s 234us/step - loss: 0.5850 - acc: 0.0000e+00
Epoch 37/100
1038/1038 [==============================] - 0s 238us/step - loss: 0.5828 - acc: 0.0000e+00
Epoch 38/100
1038/1038 [==============================] - 0s 218us/step - loss: 0.5845 - acc: 0.0000e+00
Epoch 39/100
1038/1038 [==============================] - 0s 209us/step - loss: 0.5786 - acc: 0.0000e+00
Epoch 40/100
1038/1038 [==============================] - 0s 236us/step - loss: 0.5742 - acc: 0.0000e+00
Epoch 41/100
1038/1038 [==============================] - 0s 236us/s

1038/1038 [==============================] - 0s 240us/step - loss: 0.6546 - acc: 0.0000e+00
Epoch 10/100
1038/1038 [==============================] - 0s 243us/step - loss: 0.6453 - acc: 0.0000e+00
Epoch 11/100
1038/1038 [==============================] - 0s 234us/step - loss: 0.6368 - acc: 0.0000e+00
Epoch 12/100
1038/1038 [==============================] - 0s 223us/step - loss: 0.6317 - acc: 0.0000e+00
Epoch 13/100
1038/1038 [==============================] - 0s 211us/step - loss: 0.6242 - acc: 0.0000e+00
Epoch 14/100
1038/1038 [==============================] - 0s 208us/step - loss: 0.6189 - acc: 0.0000e+00
Epoch 15/100
1038/1038 [==============================] - 0s 221us/step - loss: 0.6192 - acc: 0.0000e+00
Epoch 16/100
1038/1038 [==============================] - 0s 241us/step - loss: 0.6168 - acc: 0.0000e+00
Epoch 17/100
1038/1038 [==============================] - 0s 220us/step - loss: 0.6153 - acc: 0.0000e+00
Epoch 18/100
1038/1038 [==============================] - 0s 210us/s

1038/1038 [==============================] - 0s 220us/step - loss: 0.5284 - acc: 0.0000e+00
Epoch 87/100
1038/1038 [==============================] - 0s 209us/step - loss: 0.5338 - acc: 0.0000e+00
Epoch 88/100
1038/1038 [==============================] - 0s 223us/step - loss: 0.5327 - acc: 0.0000e+00
Epoch 89/100
1038/1038 [==============================] - 0s 240us/step - loss: 0.5285 - acc: 0.0000e+00
Epoch 90/100
1038/1038 [==============================] - 0s 214us/step - loss: 0.5261 - acc: 0.0000e+00
Epoch 91/100
1038/1038 [==============================] - 0s 214us/step - loss: 0.5299 - acc: 0.0000e+00
Epoch 92/100
1038/1038 [==============================] - 0s 208us/step - loss: 0.5407 - acc: 0.0000e+00
Epoch 93/100
1038/1038 [==============================] - 0s 233us/step - loss: 0.5333 - acc: 0.0000e+00
Epoch 94/100
1038/1038 [==============================] - 0s 246us/step - loss: 0.5274 - acc: 0.0000e+00
Epoch 95/100
1038/1038 [==============================] - 0s 220us/s

1038/1038 [==============================] - 0s 266us/step - loss: 0.5651 - acc: 0.0000e+00
Epoch 64/100
1038/1038 [==============================] - 0s 217us/step - loss: 0.5625 - acc: 0.0000e+00
Epoch 65/100
1038/1038 [==============================] - 0s 228us/step - loss: 0.5692 - acc: 0.0000e+00
Epoch 66/100
1038/1038 [==============================] - 0s 246us/step - loss: 0.5626 - acc: 0.0000e+00
Epoch 67/100
1038/1038 [==============================] - 0s 232us/step - loss: 0.5663 - acc: 0.0000e+00
Epoch 68/100
1038/1038 [==============================] - 0s 230us/step - loss: 0.5640 - acc: 0.0000e+00 0s - loss: 0.5789 - acc: 0.0000e+
Epoch 69/100
1038/1038 [==============================] - 0s 251us/step - loss: 0.5563 - acc: 0.0000e+00
Epoch 70/100
1038/1038 [==============================] - 0s 247us/step - loss: 0.5566 - acc: 0.0000e+00
Epoch 71/100
1038/1038 [==============================] - 0s 238us/step - loss: 0.5566 - acc: 0.0000e+00
Epoch 72/100
1038/1038 [==========

In [55]:
accuracy

array([-0.62488843, -0.64452752, -0.69837745, -0.70294075, -0.69695107])

### Neural Net Predictions

In [56]:
test_predictions_2=model.predict(test_data)

In [57]:
actual_test_predictions_2=np.e**(test_predictions_2 - 1)

In [58]:
submission_file['total_cases']=actual_test_predictions_2.astype(int)
submission_file.to_csv('submission_2.csv',index=False)

It was 31. Better than Lasso.

## We can improve this score by following methods:

### 1. Use NN with stacking and NN on second layer
### 2. Use Grid Search CV with NN and Keras to search for best paramters
### 3. Perform deep dive into the features and improve features by adding new ones